# Horário Escolar

Importando bibliotecas:

In [1]:
import numpy as np
import random as rd
import pandas as pd
from typing import List
from collections import defaultdict

### 1° Cenário
1 grade do ensino médio (3° Ano do ensino médio), com 30 aulas:

In [24]:
grades = {
    '3-medio-basico'    :   {
        'arte': 1, 'biologia': 3, 'ed-fisica': 1, 'filosofia': 1, 'física': 3, 'geografia': 2, 'historia': 2,
        'ingles': 2, 'portugues': 4, 'matematica': 5, 'quimica': 4, 'sociologia': 1, 'redacao': 1
    }
}
turmasGrade = {
    '3-ano-A': '3-medio-basico',
    '3-ano-B': '3-medio-basico',
    '3-ano-C': '3-medio-basico'
}
professoresDisciplina = {
    'vangogh':      {'aulas': 5, 'disciplinas': ['arte']},
    'aristoteles':  {'aulas': 5, 'disciplinas': ['biologia']},
    'platao':       {'aulas': 5, 'disciplinas': ['ed-fisica']},
    'tales':        {'aulas': 5, 'disciplinas': ['filosofia']},
    'apolo':        {'aulas': 5, 'disciplinas': ['física']},
    'anaximandro':  {'aulas': 5, 'disciplinas': ['geografia']},
    'heraclito':    {'aulas': 5, 'disciplinas': ['historia']},
    'parmenides':   {'aulas': 5, 'disciplinas': ['ingles']},
    'socrates':     {'aulas': 5, 'disciplinas': ['portugues']},
    'pitagoras':    {'aulas': 5, 'disciplinas': ['matemática']},
    'democrito':    {'aulas': 5, 'disciplinas': ['quimica']},
    'epicuro':      {'aulas': 5, 'disciplinas': ['sociologia']},
    'zenao':        {'aulas': 5, 'disciplinas': ['redacao']}
}

In [25]:
def check_fit(matrix:np.ndarray, sequence:int, extremity=False) -> List[tuple]:
    '''Verifica o espaço disponível para encaixe da aula.

    Parâmetros
    ----------
    matrix: Matriz da verificação com qualquer dimensões.
    sequence: Sequência da aulas. Ex: 3 aulas de matemática sequênciais.
    extremity: Explora somente os encaixes nas extremidades.
    Ex: 3 aulas sequênciais em 5 aulas -> [(0, 0), (0, 2)]
    a possibilidade das 3 aulas no meio serão excluídas: [(0, 1)].

    Retorno
    -------
    Um array de tuplas com as posições possíveis para o encaixe.
    '''
    temp = []
    for x, y in np.ndindex(matrix.shape):
        pos = y - (sequence - 1)
        if pos < 0:
            pos = 0
        if y >= (sequence -1):
            test = np.full((1, sequence), x)[0]
            if np.all(matrix[test, list(range(pos,y+1))]):
                temp.append((x, pos))
    if extremity:
        newTemp = []
        flat = defaultdict(list)
        for x, y in temp:
            flat[x].append(y)
        for i in flat:
            newTemp.append((i, flat[i][0]))
            if len(flat[i]) > 1 and flat[i][-1]:
                newTemp.append((i, flat[i][-1]))
        return newTemp
    return temp

In [4]:
def fit(matrix:np.ndarray, sequence:int, extremity=False, random=False, oldidx=[]) -> int:
    '''Encaixa as aulas na matriz e retorna maior igual a 0 (zero) caso sucesso'''
    options = check_fit(matrix, sequence, extremity=extremity)
    if options:
        newOptions = []
        for i in options:
            if i[0] not in oldidx:
                newOptions.append(i)
        if not newOptions:
            return -1
        fitting = newOptions[0]
        if random:
            fitting = newOptions[rd.randint(0, len(newOptions) - 1)]
        matrix[
            fitting[0], 
            fitting[1]:fitting[1] + sequence
        ] = 0
        return fitting[0]
    else:
        return -1
    
def fit_transform(matrix:np.ndarray, total:int, sequence:int, extremity=True, random=True) -> bool:
    '''Encaixa todas as aulas do professor e caso não consiga retorna Falso'''
    oldidx = []
    for _ in range(int(total / sequence)):
        oldidx.append(fit(matrix, sequence, extremity, random, oldidx))
        if -1 in oldidx:
            return False
    return True

## Testes unitários

- 6 Aulas de português, com sequência de 2, nas extremidades.
- 6 Aulas de matemática, com sequência de 3. Não aleatório.

In [6]:
matrix = np.ones((5,5), int)
assert fit_transform(matrix, 6, 2) == True
assert fit_transform(matrix, 6, 3, extremity=False, random=False) == True
matrix

array([[0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0],
       [1, 1, 1, 0, 0],
       [0, 0, 1, 1, 1],
       [1, 1, 1, 1, 1]])

Mais combinações:

In [251]:
matrix = np.ones((2,5), int)
assert checkFit(matrix, 1) == [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4)]
assert checkFit(matrix, 2) == [(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3)]
assert checkFit(matrix, 3) == [(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2)]
assert checkFit(matrix, 4) == [(0, 0), (0, 1), (1, 0), (1, 1)]
assert checkFit(matrix, 5) == [(0, 0), (1, 0)]
assert checkFit(np.array([
[1, 1, 1, 0, 1],
[0, 0, 0, 1, 0],
[1, 0, 0, 0, 1],
[1, 1, 1, 0, 0]]), 2) == [(0, 0), (0, 1), (3, 0), (3, 1)]
assert checkFit(matrix, 1, True) == [(0, 0), (0, 4), (1, 0), (1, 4)]
assert checkFit(matrix, 2, True) == [(0, 0), (0, 3), (1, 0), (1, 3)]
assert checkFit(matrix, 3, True) == [(0, 0), (0, 2), (1, 0), (1, 2)]
assert checkFit(matrix, 4, True) == [(0, 0), (0, 1), (1, 0), (1, 1)]
assert checkFit(matrix, 5, True) == [(0, 0), (1, 0)]
assert checkFit(np.array([
[1, 1, 1, 0, 1],
[0, 0, 0, 1, 0],
[1, 0, 0, 0, 1],
[1, 1, 1, 0, 0]]), 2, True) == [(0, 0), (0, 1), (3, 0), (3, 1)]

In [7]:
# def transform(matrix:np.ndarray, classes:list):
#     count = 0
#     while count < matrix.size:
#         print('tentativa,', count)
#         success = True
#         mcopy = np.ones((5,5), int)
#         for i,c in enumerate(classes):
#             if not fit(mcopy, c):
#                 #print('false', mcopy)
#                 success = False
#                 print(count, '| Não foi possível encaixar', c, 'aulas, na posição', i)
#                 break
#         #print('ok', mcopy)
#         if success:
#             print('ok')
#             print(mcopy)
#             #return mcopy
#         count += 1
    
# aulas = [2,2,3,3,3,1,1,2,3,2,1,2]
# matrix = np.ones((5,5), int)
# print(matrix.size, sum(aulas), len(aulas))
# transform(matrix, aulas)
#matrix
#df = pd.DataFrame(matrix, columns=['Seg', 'Ter', 'Qua', 'Qui', 'Sex'])
#df